## 卷积层

### torch.nn 中的卷积层与 torch.nn.functional 差别
torch.nn.functional 中的方法常使用在 forward 内部。以卷积层举例，如果把 F.conv2d 中的参数 weights 先使用 nn.Parameters
在 __init__ 方法中进行注册，然后在 forward 内部调用 F.conv2d，这个操作和直接使用 nn.conv2d 是等同的。  

换句话说，torch.nn 中的 Layer 将参数直接封装好了，在 __init__ 中初始化中直接将参数进行注册，
而 torch.nn.functional 则将参数开放了出来，完全可以用做一个固定的 tensor 不进行更新。

In [9]:
import torch 
import torch.nn.functional as F 

### F.conv2d

In [10]:
input_t = torch.tensor([
    [1, 2, 0, 3, 1],
    [0, 1, 2, 3, 1],
    [1, 2, 1, 0, 0],
    [5, 2, 3, 1, 1],
    [2, 1, 0, 1, 1]
])

kernel = torch.tensor([
    [1, 2, 1],
    [0, 1, 0],
    [2, 1, 0]
])

input_t = input_t.reshape((1, 1, 5, 5))
kernel = kernel.reshape((1, 1, 3, 3))
print(input_t.shape)
print(kernel.shape)

output = F.conv2d(input_t, kernel, stride=1)
print(output)

output2 = F.conv2d(input_t, kernel, stride=2)
print(output2)

output3 = F.conv2d(input_t, kernel, stride=1, padding=1)
print(output3)


torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 3, 3])
tensor([[[[10, 12, 12],
          [18, 16, 16],
          [13,  9,  3]]]])
tensor([[[[10, 12],
          [13,  3]]]])
tensor([[[[ 1,  3,  4, 10,  8],
          [ 5, 10, 12, 12,  6],
          [ 7, 18, 16, 16,  8],
          [11, 13,  9,  3,  4],
          [14, 13,  9,  7,  4]]]])


### nn.conv2d

In [16]:
import torchvision
from torch.utils.data import DataLoader
import torch.nn as nn 
from torch.utils.tensorboard import SummaryWriter

dataset = torchvision.datasets.CIFAR10("./_data", train=False, 
            transform=torchvision.transforms.ToTensor(), download=True)
dataloader = DataLoader(dataset, batch_size=64)

class Foo(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=6, kernel_size=3, stride=1, padding=0)

    def forward(self, x):
        x = self.conv1(x)
        return x 

foo = Foo()

imgs, labels = next(iter(dataloader)) 
output = foo(imgs)
print(imgs.shape)
print(output.shape)

writor = SummaryWriter("./_log/conv")
writor.add_images("input", imgs)

# output 有6个channel，无法进行
writor.add_images("output", output.reshape((-1, 3, 30, 30)))
writor.close()

Files already downloaded and verified
torch.Size([64, 3, 32, 32])
torch.Size([64, 6, 30, 30])


## 池化层
池化层默认kernel移动的方式与卷积层不同，池化层的kernel默认移动时要保证互不相交。
stride=kernel_size

In [21]:
input_t = torch.tensor([
    [1, 2, 0, 3, 1],
    [0, 1, 2, 3, 1],
    [1, 2, 1, 0, 0],
    [5, 2, 3, 1, 1],
    [2, 1, 0, 1, 1]
], dtype=torch.float32)

input_t = input_t.reshape((-1, 1, 5, 5))
print(input_t.shape)

class Foo(nn.Module):

    def __init__(self):
        super().__init__()
        self.maxpool1 = nn.MaxPool2d(3, ceil_mode=True)

    def forward(self, x):
        output = self.maxpool1(x)
        return output

foo = Foo()
print(foo(input_t))


torch.Size([1, 1, 5, 5])
tensor([[[[2., 3.],
          [5., 1.]]]])


In [23]:
dataset = torchvision.datasets.CIFAR10("./_data", train=False, 
            transform=torchvision.transforms.ToTensor(), download=False)

dataloader = DataLoader(dataset, batch_size=64)

imgs, labels = next(iter(dataloader))
writor = SummaryWriter("./_log/maxpool")
writor.add_images("input", imgs)
writor.add_images("output", foo(imgs))
writor.close()